# Bicubic interpolation

Interpolating data points on two-dimensional regular grid. The interpolated surface is smoother than the corresponding surfaces obtained by bilinear interpolation. Bicubic interpolation is achieved by spline functions provided by GSL.

> **Warning** When using this interpolator, pay attention to the undefined values.
> Because as long as the calculation window uses an indefinite point, the
> interpolator will generate indefinite values. This means that this
> interpolator increases the area covered by the masked values. To avoid this
> behavior, it is necessary to fill_values the grid to delete undefined values.

Let's start by building our interpolator:

In [ ]:
import xarray as xr
import pyinterp.backends.xarray as pxr

ds = xr.load_dataset("../tests/dataset/mss.nc")
interpolator = pxr.Grid2D(ds.data_vars["mss"])
interpolator

We will build a new grid that will be used to build a new interpolated grid.

In [ ]:
import numpy as np

# The coordinates used for interpolation are shifted to avoid using the
# points of the bivariate function.
mx, my = np.meshgrid(np.arange(-180, 180, 1) + 1 / 3.0,
                     np.arange(-89, 89, 1) + 1 / 3.0,
                     indexing='ij')

The [bicubic](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.bicubic.html#pyinterp.bicubic.bicubic) interpolation function has more parameters in order to define the data frame used by the spline functions and how to process the edges of the regional grids:

In [ ]:
import pyinterp.bicubic

mss = interpolator.bicubic(dict(lon=mx.flatten(), lat=my.flatten())).reshape(mx.shape)

Let's visualize our result and the original grid.

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
%matplotlib inline

fig = plt.figure(figsize=(18, 9))
ax = fig.add_subplot(121, projection=ccrs.PlateCarree(central_longitude=180))
lons, lats = np.meshgrid(interpolator.x, interpolator.y, indexing='ij')
ax.pcolormesh(lons, lats, interpolator.array, cmap='jet',
              transform=ccrs.PlateCarree())
ax.coastlines()
ax.set_title("Original MSS")

ax = fig.add_subplot(122, projection=ccrs.PlateCarree())
ax.pcolormesh(mx, my, mss, cmap='jet',
              transform=ccrs.PlateCarree())
ax.coastlines()
ax.set_title("Interpolated MSS")